In [ ]:
## W261 Week5 HW

#### Names Safyre Anderson, Howard Wen , Vamsi Sakhamuri

#### Emails safyre@berkeley.edu, howard.wen1@gmail.com, vamsi@ischool.berkeley.edu

#### Time of Initial Submission: February 18th, 2016 8am PST

#### Section W261-3, Spring 2016

#### Week 5 Homework

### HW 5.0
*What is a data warehouse? What is a Star schema? When is it used?*

1. *Data Warehouse:*

https://en.wikipedia.org/wiki/Data_warehouse. A data warehouse is an enterprise-level repository for historical and current datasets. Typically, a business has many sources of data that need to be processed for a variety of applications: e.g. reporting, business analyics, and/ or predictive analytics.  For example, in mobile advertising data pipelines, an ad request from an app on a customer's mobile device will require a decision that involves streaming usage data in addition to historical data. The historical data would have been used to determine which types of ads are relevant to the end user, while the streaming data used to generate a prediction. Data collected from these different sources and various purposes need to be passed through some form of transformation and storage in order to be processed in subsequent parts of the pipeline. The gathering of data, transforming them and storing them is known as ETL or extract, transform, and load. The loading piece of ETL refers to storing processed data into the data warehouse--typically, a relational database, but can also include semi-stuctured and unstructured data storage.


Two traditional examples of data warehousing strategies are OLAP (online analytical processing) and OLTP (online transactional processing).

2. *Star Schema:*

The star schema is a framework for organizing data in a relational database where a central fact table acts as a means to connect data between different dimensional tables. The fact table will contain foreign keys for each of the branching tables. Additionally, each of the branching tables focusses on data related to one dimension that an analysis could splice on. For example an ecommerce schema might have a transactions-based fact table that contains foreign keys for a customers table, and a payment table. The transactions table will be able to join with both the customers and payment tables directly, but the payment table and customer table would need to join with each other indirectly through the transactions table since they would not (in this example) contain each others foreign keys. 

3. *When is it used?:*

The star schema is useful for highly structured data that may have numerous dimensions that could be used to break down the data during analyses. The main reason this is useful in most reporting applications is because it makes querying data much more efficient. Instead of having one large table with all the data and many rows, joining a smaller table with a larger one can shrink the size of the data significantly before it is extracted.


### HW 5.1
1. *In the database world What is 3NF? Does machine learning use data in 3NF? If so why? In what form does ML consume data?*

3NF stands for "Third normal form". This refers to how data are organized with their respective tables and keys: Attributes in a table should provide a fact about the table's key and nothing else ("Nothing but the key" https://en.wikipedia.org/wiki/Third_normal_form). This structure allows a database to avoid duplicating data as much as possible and therefore save storage space. Machine learning would not be able to utilize data in this form very well--features would be scattered across different sources so the classifier would not be able to train on features in different tables. ML classifiers need to have all the data they depend on in one location (distributed or not) and therefore need the data to be denormalized.

2. *Why would one use log files that are denormalized?*

Log files are normally stored as semi-structured or unstructured data. They may have some regular format, but for the most part, log files contain plain English that would need to be parsed during analysis. If log files were placed into a structure and normalized, closely related pieces of information that could be useful would be separated from each other--for instance, an error message with a time stamp. Furthermore, a denormalized log file is more usable because log files are all text. Normalized data typically contains data that can be strongly typed, but if a log file is more intuitively parsed as a document then normalization may just overcomplicate things.



### HW 5.2
*Using MRJob, implement a hashside join (memory-backed map-side) for left, 
right and inner joins. Run your code on the  data used in HW 4.4: (Recall HW 4.4: Find the most frequent visitor of each page using mrjob and the output of 4.2  (i.e., transfromed log file). In this output please include the webpage URL, webpageID and Visitor ID.)*
:

Justify which table you chose as the Left table in this hashside join.

Please report the number of rows resulting from:

(1) Left joining Table Left with Table Right
(2) Right joining Table Left with Table Right
(3) Inner joining Table Left with Table Right

In [2]:
## first get 4.2 data and preprocess it:
!wget -O anonymous-msweb.data http://archive.ics.uci.edu/ml/machine-learning-databases/anonymous/anonymous-msweb.data

## python code to reformat the data:
# check format of whole file
#!cat anonymous-msweb.data
write_file_loc = "msweb_processed.txt"
writefile = open(write_file_loc, 'wb')

with open("anonymous-msweb.data", 'rU') as f:
    for line in f.readlines():
        line = line.strip().split(',')
        
        # skip all the header rows
        if line[0] not in ['C', 'V']:
            continue
            
        if line[0] == 'C':
            user_id = line[2]
        
        elif line[0] == 'V':
            writefile.write('V,{},1,C,{}\n'.format(line[1], user_id))
        
        else:
            continue

--2016-02-15 14:26:38--  http://archive.ics.uci.edu/ml/machine-learning-databases/anonymous/anonymous-msweb.data
Resolving archive.ics.uci.edu... 128.195.10.249
Connecting to archive.ics.uci.edu|128.195.10.249|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423098 (1.4M) [text/plain]
Saving to: 'anonymous-msweb.data'

anonymous-msweb.dat 100%[=====================>]   1.36M  1.91MB/s   in 0.7s   

2016-02-15 14:26:39 (1.91 MB/s) - 'anonymous-msweb.data' saved [1423098/1423098]



In [4]:
# preprocessed file is:
!head -n10 msweb_processed.txt

V,1000,1,C,10001
V,1001,1,C,10001
V,1002,1,C,10001
V,1001,1,C,10002
V,1003,1,C,10002
V,1001,1,C,10003
V,1003,1,C,10003
V,1004,1,C,10003
V,1005,1,C,10004
V,1006,1,C,10005


In [6]:
!head -n15 anonymous-msweb.data

I,4,"www.microsoft.com","created by getlog.pl"
T,1,"VRoot",0,0,"VRoot"
N,0,"0"
N,1,"1"
T,2,"Hide1",0,0,"Hide"
N,0,"0"
N,1,"1"
A,1287,1,"International AutoRoute","/autoroute"
A,1288,1,"library","/library"
A,1289,1,"Master Chef Product Information","/masterchef"
A,1297,1,"Central America","/centroam"
A,1215,1,"For Developers Only Info","/developer"
A,1279,1,"Multimedia Golf","/msgolf"
A,1239,1,"Microsoft Consulting","/msconsult"
A,1282,1,"home","/home"


In [12]:
## now process a new set of data with just URL and webpageID for merging
## first get 4.2 data and preprocess it:
!wget -O anonymous-msweb.data http://archive.ics.uci.edu/ml/machine-learning-databases/anonymous/anonymous-msweb.data

## python code to reformat the data:
# check format of whole file
#!cat anonymous-msweb.data
write_file_loc = "msweb_dim_url.txt"
writefile = open(write_file_loc, 'wb')

with open("anonymous-msweb.data", 'rU') as f:
    for line in f.readlines():
        line = line.strip().split(',')
        
        # skip all the header rows
        if line[0] == 'A':
            # attach the relative url to the home url. Remove ""'s
            full_url = "http://www.microsoft.com"+line[4].strip('"')
            writefile.write('{url_id}\t{url}\n'.format(url_id=line[1],url=full_url))
        
        #elif line[0] == 'V':
        #    writefile.write('V,{},1,C,{}\n'.format(line[1], user_id))
        
        #else:
        #    continue

--2016-02-15 15:20:24--  http://archive.ics.uci.edu/ml/machine-learning-databases/anonymous/anonymous-msweb.data
Resolving archive.ics.uci.edu... 128.195.10.249
Connecting to archive.ics.uci.edu|128.195.10.249|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423098 (1.4M) [text/plain]
Saving to: 'anonymous-msweb.data'

anonymous-msweb.dat 100%[=====================>]   1.36M  1.88MB/s   in 0.7s   

2016-02-15 15:20:25 (1.88 MB/s) - 'anonymous-msweb.data' saved [1423098/1423098]



In [60]:
!head -n10 msweb_dim_url.txt

1287	http://www.microsoft.com/autoroute
1288	http://www.microsoft.com/library
1289	http://www.microsoft.com/masterchef
1297	http://www.microsoft.com/centroam
1215	http://www.microsoft.com/developer
1279	http://www.microsoft.com/msgolf
1239	http://www.microsoft.com/msconsult
1282	http://www.microsoft.com/home
1251	http://www.microsoft.com/referencesupport
1121	http://www.microsoft.com/magazine


In [83]:
!wc -l msweb_dim_url.txt
!wc -l msweb_processed.txt

     304 msweb_dim_url.txt
   98654 msweb_processed.txt


In [78]:
%%writefile MR_HW52_joins.py
#!/usr/bin/env python
# find the most frequent visitors per url
from mrjob.job import MRJob
from mrjob.step import MRStep
import csv
from heapq import heappush, heappop,nlargest

class MapSideJoins(MRJob):
    def steps(self):
        return [MRStep(
            mapper_init = self.mapper_init,
            mapper = self.mapper,
            mapper_final = self.mapper_final
            )]
        
    def mapper_init(self):
        # set variables
        # inspired by example on lecture slide 121
        self.URLS = {}
        
        # for right outer join
        self.URLS_ON_RIGHT = {}
        
        # initialize row counts
        self.left = 0
        self.right = 0 
        self.inner = 0
        
        # read in the page_url table
        # formatted like : 1287 http://www.microsoft.com/autoroute
        f = open("msweb_dim_url.txt", 'rU')
        for line in f:
            line = line.strip()
            data = line.split()
            #store page-id as key in URLS dictionary
            self.URLS[data[0]] = data[1]
                
    def mapper(self, _, line):
        """stream in output from 4.2 (msweb_processed.txt):
        
        V,1000,1,C,10001
        
        and match with keys in self.URLS"""
        self.increment_counter('group', 'Number_mapper_calls',1)
        line = line.strip().split(',')
        key = line[1]
        
        if key in self.URLS.keys():
            ## outputs are : JOIN_TYPE, URL_ID, URL, CUSTOMER_ID
            # map inner join
            self.inner += 1
            yield None, ("inner", key, self.URLS[key], line[4])
            
            # map visits left join urls on url_id
            # this code assumes each customer has a visit
            # if there are visits to no URLS, we will catch it in the second mapper
            # although this probably isn't the case...
            self.left += 1
            yield None, ("left", key, self.URLS[key], line[4])
            
            # map visits right join urls on url_id
            # keep track of url_ids that have not been visited
            self.right += 1
            yield None, ("right", key, self.URLS[key], line[4])
            self.URLS_ON_RIGHT[key] = self.URLS[key]
        
        #right join will also include sites that have not been visited
        # no customer_id
        else:
            self.right += 1
            yield None, ("right", key, self.URLS[key], "NA")
            
    def mapper_final(self):
        for key in self.URLS.keys():
            # in a left join, if a key exists on the left table, but not the right table
            # then we need a row with data from the left table and null values from the missing
            # data on the right
            if key not in self.URLS_ON_RIGHT.keys():
                self.left += 1
                yield None, ("left", key, "NA",  self.URLS[key])
        
        ## final outputs:
        yield None, ()
        yield None, ("left-joined rows: " + str(self.left))
        yield None, ("right-joined rows: " + str(self.right))
        yield None, ("inner-joined rows: " + str(self.inner))
    
if __name__ == "__main__":
    MapSideJoins.run()

Overwriting MR_HW52_joins.py


In [81]:
!rm hw52_out.txt
!./MR_HW52_joins.py msweb_processed.txt \
    --file msweb_dim_url.txt > hw52_out.txt

no configs found; falling back on auto-configuration
no configs found; falling back on auto-configuration
creating tmp directory /var/folders/jz/dhc2gzfj2091nhfmrbvv3cjh0000gn/T/MR_HW52_joins.Safyre.20160216.091239.013019

PLEASE NOTE: Starting in mrjob v0.5.0, protocols will be strict by default. It's recommended you run your job with --strict-protocols or set up mrjob.conf as described at https://pythonhosted.org/mrjob/whats-new.html#ready-for-strict-protocols

writing to /var/folders/jz/dhc2gzfj2091nhfmrbvv3cjh0000gn/T/MR_HW52_joins.Safyre.20160216.091239.013019/step-0-mapper_part-00000
Counters from step 1:
  group:
    Number_mapper_calls: 98654
Moving /var/folders/jz/dhc2gzfj2091nhfmrbvv3cjh0000gn/T/MR_HW52_joins.Safyre.20160216.091239.013019/step-0-mapper_part-00000 -> /var/folders/jz/dhc2gzfj2091nhfmrbvv3cjh0000gn/T/MR_HW52_joins.Safyre.20160216.091239.013019/output/part-00000
Streaming final output from /var/folders/jz/dhc2gzfj2091nhfmrbvv3cjh0000gn/T/MR_HW52_joins.Safyre.2016

In [82]:
!tail -n10 hw52_out.txt

null	["left", "1291", "NA", "http://www.microsoft.com/news"]
null	["left", "1297", "NA", "http://www.microsoft.com/centroam"]
null	["left", "1294", "NA", "http://www.microsoft.com/bookshelf"]
null	["left", "1287", "NA", "http://www.microsoft.com/autoroute"]
null	["left", "1289", "NA", "http://www.microsoft.com/masterchef"]
null	["left", "1288", "NA", "http://www.microsoft.com/library"]
null	[]
null	"left-joined rows: 98663"
null	"right-joined rows: 98654"
null	"inner-joined rows: 98654"


In [88]:
!chmod +x MR_HW52_joins.py
!python MR_HW52_joins.py -r emr s3://w261-safyre-hw52-input/msweb_processed.txt \
    --conf-path /Users/Safyre/cc-mrjob/mrjob.conf\
    --file s3://w261-safyre-hw52-input/msweb_dim_url.txt \
    --output-dir= s3://w261-safyre-hw52-output/ \
    --no-output \
    --cleanup=NONE\
    --no-strict-protocol

Got unexpected keyword arguments: ssh_tunnel
using existing scratch bucket mrjob-62fd57571f35a64e
using s3://mrjob-62fd57571f35a64e/tmp/ as our scratch dir on S3
creating tmp directory /var/folders/jz/dhc2gzfj2091nhfmrbvv3cjh0000gn/T/MR_HW52_joins.Safyre.20160216.092228.655665
writing master bootstrap script to /var/folders/jz/dhc2gzfj2091nhfmrbvv3cjh0000gn/T/MR_HW52_joins.Safyre.20160216.092228.655665/b.py
Copying non-input files into s3://mrjob-62fd57571f35a64e/tmp/MR_HW52_joins.Safyre.20160216.092228.655665/files/
Waiting 5.0s for S3 eventual consistency
Creating Elastic MapReduce job flow
Job flow created with ID: j-1TOO3XZPEKZ4R
Created new job flow j-1TOO3XZPEKZ4R
Job launched 30.5s ago, status STARTING: Provisioning Amazon EC2 capacity
Job launched 61.5s ago, status STARTING: Provisioning Amazon EC2 capacity
Job launched 92.0s ago, status STARTING: Provisioning Amazon EC2 capacity
Job launched 122.6s ago, status STARTING: Provisioning Amazon EC2 capacity
Job launched 153.1s ago,

### HW 5.3  EDA of Google n-grams dataset
A large subset of the Google n-grams dataset

https://aws.amazon.com/datasets/google-books-ngrams/

which we have placed in a bucket/folder on Dropbox on s3:

   https://www.dropbox.com/sh/tmqpc4o0xswhkvz/AACUifrl6wrMrlK6a3X3lZ9Ea?dl=0 

   `s3://filtered-5grams/`

In particular, this bucket contains (~200) files (10Meg each) in the format:

	(ngram) \t (count) \t (pages_count) \t (books_count)

For HW 5.3-5.5, for the Google n-grams dataset unit test and regression test your code using the 
first 10 lines of the following file:

googlebooks-eng-all-5gram-20090715-0-filtered.txt

Once you are happy with your test results proceed to generating  your results on the Google n-grams dataset. 

Do some EDA on this dataset using mrjob, e.g., 

- Longest 5-gram (number of characters)
- Top 10 most frequent words (please use the count information), i.e., unigrams
- 20 Most/Least densely appearing words (count/pages_count) sorted in decreasing order of relative frequency 
- Distribution of 5-gram sizes (character length).  E.g., count (using the count field) up how many times a 5-gram of 50 characters shows up. Plot the data graphically using a histogram.

In [47]:
## download dev data
!wget -O google-5grams-test.txt "https://www.dropbox.com/sh/tmqpc4o0xswhkvz/AACr50woxiBWoaiiLmnwduX8a/googlebooks-eng-all-5gram-20090715-0-filtered.txt?dl=0"

--2016-02-15 23:42:40--  https://www.dropbox.com/sh/tmqpc4o0xswhkvz/AACr50woxiBWoaiiLmnwduX8a/googlebooks-eng-all-5gram-20090715-0-filtered.txt?dl=0
Resolving www.dropbox.com... 108.160.172.206, 108.160.172.238
Connecting to www.dropbox.com|108.160.172.206|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://dl.dropboxusercontent.com/content_link/aIZVHfoBfzACzWxyagqbBmxeGTl601USgFACattc1MpkSXDxZ9gk5hMyZTQFqWBn/file [following]
--2016-02-15 23:42:41--  https://dl.dropboxusercontent.com/content_link/aIZVHfoBfzACzWxyagqbBmxeGTl601USgFACattc1MpkSXDxZ9gk5hMyZTQFqWBn/file
Resolving dl.dropboxusercontent.com... 199.47.217.69
Connecting to dl.dropboxusercontent.com|199.47.217.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11444614 (11M) [text/plain]
Saving to: 'google-5grams-test.txt'

google-5grams-test. 100%[=====================>]  10.91M  1.03MB/s   in 8.3s   

2016-02-15 23:42:50 (1.31 MB/s) - 'google-5grams-test.txt' saved [

In [48]:
#ngram \t count \t page_count \t book_count
!head -n10 google-5grams-test.txt

A BILL FOR ESTABLISHING RELIGIOUS	59	59	54
A Biography of General George	92	90	74
A Case Study in Government	102	102	78
A Case Study of Female	447	447	327
A Case Study of Limited	55	55	43
A Child's Christmas in Wales	1099	1061	866
A Circumstantial Narrative of the	62	62	50
A City by the Sea	62	60	49
A Collection of Fairy Tales	123	117	80
A Collection of Forms of	116	103	82


In [77]:
## upload test set to s3 bucket
import boto
from boto.s3.connection import S3Connection
from boto.s3.key import Key

# AWS_ACCESS_KEY_ID and AWS_SECRET_ACCESS_KEY are environment variables
# S3Connection argument can be empty
conn = S3Connection()
bucket = conn.create_bucket('w261-safyre-hw53-input')  # sub-datasets bucket already exists
myBucket = conn.get_bucket('w261-safyre-hw53-input')

k = Key(myBucket)
k.key = 'google-5grams-test.txt'
k.set_contents_from_filename('google-5grams-test.txt')
#googlebooks-eng-all-5gram-20090715-0-filtered.txt

11444614

In [ ]:
# mapper for 5.3 word count
%%writefile MR_HW53b.py
#!/usr/bin/env python
# find the most frequent visitors per url
from mrjob.job import MRJob
from mrjob.step import MRStep
import csv
from heapq import heappush, heappop,nlargest
import re

class WordCount(MRJob):
    def steps(self):
        return [MRStep(
            mapper_init = self.mapper_init,
            mapper = self.mapper,
            combiner = self.combiner,
            reducer = self.reducer
            ),
            MRStep(
            reducer = self.reducer_2nd
            )]
        
    def mapper_init(self):
        self.counts = {}
    
    def mapper(self, _, line):
        line.strip()
        [ngram, gram_count,pages,books] = re.split("\t",line)
        gram_count = int(gram_count)
        words = re.split(" ",ngram)
        for word in words:
            self.counts.setdefault(word,0)
            self.counts[word] += gram_count
        for word in counts.keys():
            yield word, counts[word]
                
    def combiner(self, word, counts):
        total = sum(counts)
        yield word, total
    
    def reducer(self, word, total):
        total = sum(counts)
        yield None,(word, total)
        
    def reducer_2nd(self, _, word_count_pairs):
        heap = []
        for p in word_count_pairs:
            heappush(heap,p)
        top10 = nlargest(10, heap)
        yield None,top10

In [ ]:
!chmod +x MR_HW53b.py
!./MR_HW53b.py google-5grams-test.txt > hw53b_out.txt

In [ ]:
# mapper for 5.3 ngrams EDA
%%writefile MR_HW53acd.py
#!/usr/bin/env python
from mrjob.job import MRJob
from mrjob.step import MRStep
import csv
from heapq import heappush, heappop,nlargest
import re

class WordCount(MRJob):
    def steps(self):
        return [MRStep(
            mapper_init = self.mapper_init,
            mapper = self.mapper,
            mapper_final = self.mapper_final
            )]
        
    def mapper_init(self):
        self.counts = {}
    
    def mapper(self, _, line):
        line.strip()
        [ngram, gram_count,pages,books] = re.split("\t",line)
        gram_count = int(gram_count)
        words = re.split(" ",ngram)
        for word in words:
            self.counts.setdefault(word,0)
            self.counts[word] += gram_count
        for word in counts.keys():
            yield word, counts[word]
                
    def combiner(self, word, counts):
        total = sum(counts)
        yield word, total
    
    def reducer(self, word, total):
        total = sum(counts)
        yield None,(word, total)
        
    def reducer_2nd(self, _, word_count_pairs):
        heap = []
        for p in word_count_pairs:
            heappush(heap,p)
        top10 = nlargest(10, heap)
        yield None,top10